In [ ]:
from keras.models import Sequential
from keras.layers import Dense 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import roc_curve
from keras.utils import to_categorical
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

In [ ]:
def decode(datum):
    return np.argmax(datum)

In [ ]:
dic={'Male':1,'Female':0,'Yes':1,'No':0,'Positive':1,'Negative':0}

In [ ]:
# We are reading our data
df = pd.read_csv("../input/early-stage-diabetes-risk-prediction-datasets/diabetes_data_upload.csv")
print(df.head(),df.shape)

df.boxplot()

In [ ]:
# First 5 rows of our data
df.head()

In [ ]:
all_col=df.columns

for i in range(df.shape[1]):
    df.replace
print(all_col[-1],len(all_col))
for col in range(1,len(all_col)):
    df=df.replace({all_col[col]: dic})
print(df.head())

In [ ]:
df['class'].value_counts()

In [ ]:
sns.countplot(x="class", data=df, palette="bwr")
plt.show()

In [ ]:
countNo = len(df[df['class'] == 0])
countYes = len(df[df['class'] == 1])
print("Percentage of NO: {:.2f}%".format((countNo / (len(df['class']))*100)))
print("Percentage of Yes: {:.2f}%".format((countYes / (len(df['class'])*100))))

In [ ]:
y = df['class'].values
x_data = df.drop(['class'], axis = 1)

In [ ]:
# Normalize
x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=0)

In [ ]:
#transpose matrices
x_train = x_train.T
y_train = y_train.T
x_test = x_test.T
y_test = y_test.T

In [ ]:
#initialize
def initialize(dimension):
    
    weight = np.full((dimension,1),0.01)
    bias = 0.0
    return weight,bias

In [ ]:
def sigmoid(z):
    
    y_head = 1/(1+ np.exp(-z))
    return y_head

In [ ]:
def forwardBackward(weight,bias,x_train,y_train):
    # Forward
    
    y_head = sigmoid(np.dot(weight.T,x_train) + bias)
    loss = -(y_train*np.log(y_head) + (1-y_train)*np.log(1-y_head))
    cost = np.sum(loss) / x_train.shape[1]
    
    # Backward
    derivative_weight = np.dot(x_train,((y_head-y_train).T))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"Derivative Weight" : derivative_weight, "Derivative Bias" : derivative_bias}
    
    return cost,gradients

In [ ]:
def update(weight,bias,x_train,y_train,learningRate,iteration) :
    costList = []
    index = []
    
    #for each iteration, update weight and bias values
    for i in range(iteration):
        cost,gradients = forwardBackward(weight,bias,x_train,y_train)
        weight = weight - learningRate * gradients["Derivative Weight"]
        bias = bias - learningRate * gradients["Derivative Bias"]
        
        costList.append(cost)
        index.append(i)

    parameters = {"weight": weight,"bias": bias}
    
    print("iteration:",iteration)
    print("cost:",cost)

    plt.plot(index,costList)
    plt.xlabel("Number of Iteration")
    plt.ylabel("Cost")
    plt.show()

    return parameters, gradients

In [ ]:
def predict(weight,bias,x_test):
    z = np.dot(weight.T,x_test) + bias
    y_head = sigmoid(z)

    y_prediction = np.zeros((1,x_test.shape[1]))
    
    for i in range(y_head.shape[1]):
        if y_head[0,i] <= 0.5:
            y_prediction[0,i] = 0
        else:
            y_prediction[0,i] = 1
    return y_prediction

In [ ]:
def logistic_regression(x_train,y_train,x_test,y_test,learningRate,iteration):
    dimension = x_train.shape[0]
    weight,bias = initialize(dimension)
    
    parameters, gradients = update(weight,bias,x_train,y_train,learningRate,iteration)

    y_prediction = predict(parameters["weight"],parameters["bias"],x_test)
    
    print("Manuel Test Accuracy: {:.2f}%".format((100 - np.mean(np.abs(y_prediction - y_test))*100)))

In [ ]:
logistic_regression(x_train,y_train,x_test,y_test,1,100)

**Sklearn Logistic Regression**

In [ ]:
accuracies = {}

lr = LogisticRegression()
lr.fit(x_train.T,y_train.T)
acc = lr.score(x_test.T,y_test.T)*100

accuracies['Logistic Regression'] = acc
print("Test Accuracy {:.2f}%".format(acc))

K-Nearest Neighbour (KNN) Classification

In [ ]:
# KNN Model
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 2)  # n_neighbors means k
knn.fit(x_train.T, y_train.T)
prediction = knn.predict(x_test.T)

print("{} NN Score: {:.2f}%".format(2, knn.score(x_test.T, y_test.T)*100))

In [ ]:
# try ro find best k value
scoreList = []
for i in range(1,20):
    knn2 = KNeighborsClassifier(n_neighbors = i)  # n_neighbors means k
    knn2.fit(x_train.T, y_train.T)
    scoreList.append(knn2.score(x_test.T, y_test.T))
    
plt.plot(range(1,20), scoreList)
plt.xticks(np.arange(1,20,1))
plt.xlabel("K value")
plt.ylabel("Score")
plt.show()

acc = max(scoreList)*100
accuracies['KNN'] = acc
print("Maximum KNN Score is {:.2f}%".format(acc))

Support Vector Machine (SVM) Algorithm

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(random_state = 1,kernel='linear')
svm.fit(x_train.T, y_train.T)

acc = svm.score(x_test.T,y_test.T)*100
accuracies['SVM'] = acc
print("Test Accuracy of SVM Algorithm: {:.2f}%".format(acc))

Naive Bayes Algorithm

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train.T, y_train.T)

acc = nb.score(x_test.T,y_test.T)*100
accuracies['Naive Bayes'] = acc
print("Accuracy of Naive Bayes: {:.2f}%".format(acc))

Decision Tree Algorithm

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train.T, y_train.T)

acc = dtc.score(x_test.T, y_test.T)*100
accuracies['Decision Tree'] = acc
print("Decision Tree Test Accuracy {:.2f}%".format(acc))

Random Forest Classification

In [ ]:
# Random Forest Classification
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 2, random_state = 1)
rf.fit(x_train.T, y_train.T)

acc = rf.score(x_test.T,y_test.T)*100
accuracies['Random Forest'] = acc
print("Random Forest Algorithm Accuracy Score : {:.2f}%".format(acc))

Comparing Models

In [ ]:
colors = ["purple", "green", "orange", "magenta","#CFC60E","#0FBBAE"]

sns.set_style("whitegrid")
plt.figure(figsize=(16,5))
plt.yticks(np.arange(0,100,10))
plt.ylabel("Accuracy %")
plt.xlabel("Algorithms")
sns.barplot(x=list(accuracies.keys()), y=list(accuracies.values()), palette=colors)
plt.show()

## Confusion Matrix, F1 Score

In [ ]:
# Predicted values
y_head_lr = lr.predict(x_test.T)
knn3 = KNeighborsClassifier(n_neighbors = 3)
knn3.fit(x_train.T, y_train.T)
y_head_knn = knn3.predict(x_test.T)
y_head_svm = svm.predict(x_test.T)
y_head_nb = nb.predict(x_test.T)
y_head_dtc = dtc.predict(x_test.T)
y_head_rf = rf.predict(x_test.T)

In [ ]:
from sklearn.metrics import f1_score
f1_lr = f1_score(y_test,y_head_lr)
f1_knn = f1_score(y_test,y_head_knn)
f1_svm = f1_score(y_test,y_head_svm)
f1_nb = f1_score(y_test,y_head_nb)
f1_dtc = f1_score(y_test,y_head_dtc)
f1_rf = f1_score(y_test,y_head_rf)

print('Logistic Regression F1 score:', f1_lr)
print('K Nearest Neighbors F1 score:', f1_knn)
print('Support Vector Machine F1 score:', f1_svm)
print('Naive Bayes F1 score:', f1_nb)
print('Decision Tree s F1 score:', f1_dtc)
print('Random Forest F1 score:', f1_rf)

In [ ]:
from sklearn.metrics import recall_score
rc_lr = recall_score(y_test,y_head_lr)
rc_knn = recall_score(y_test,y_head_knn)
rc_svm = recall_score(y_test,y_head_svm)
rc_nb = recall_score(y_test,y_head_nb)
rc_dtc = recall_score(y_test,y_head_dtc)
rc_rf = recall_score(y_test,y_head_rf)

print('Logistic Regression recall:', rc_lr)
print('K Nearest Neighbors recall:', rc_knn)
print('Support Vector Machine recall:', rc_svm)
print('Naive Bayes recall:', rc_nb)
print('Decision Tree recall:', rc_dtc)
print('Random Forest recall:', rc_rf)

In [ ]:
from sklearn.metrics import precision_score
precision_lr = precision_score(y_test,y_head_lr)
precision_knn = precision_score(y_test,y_head_knn)
precision_svm = precision_score(y_test,y_head_svm)
precision_nb = precision_score(y_test,y_head_nb)
precision_dtc = precision_score(y_test,y_head_dtc)
precision_rf = precision_score(y_test,y_head_rf)

print('Logistic Regression precision:', precision_lr)
print('K Nearest Neighbors precision:', precision_knn)
print('Support Vector Machine precision:', precision_svm)
print('Naive Bayes precision:', precision_nb)
print('Decision Tree precision:', precision_dtc)
print('Random Forest precision:', precision_rf)

In [ ]:
from sklearn.metrics import confusion_matrix

cm_lr = confusion_matrix(y_test,y_head_lr)
cm_knn = confusion_matrix(y_test,y_head_knn)
cm_svm = confusion_matrix(y_test,y_head_svm)
cm_nb = confusion_matrix(y_test,y_head_nb)
cm_dtc = confusion_matrix(y_test,y_head_dtc)
cm_rf = confusion_matrix(y_test,y_head_rf)


In [ ]:
def plot_roc_curve(false_positive_rate, true_positive_rate, label=None):
    plt.plot(false_positive_rate, true_positive_rate, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'r', linewidth=4)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate (FPR)', fontsize=16)
    plt.ylabel('True Positive Rate (TPR)', fontsize=16)
    plt.legend

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_head_lr)
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_head_knn)
fpr_svm, tpr_svm, thresholds_svm = roc_curve(y_test, y_head_svm)
fpr_nb, tpr_nb, thresholds_nb = roc_curve(y_test, y_head_nb)
fpr_dtc, tpr_dtc, thresholds_dtc = roc_curve(y_test, y_head_dtc)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_head_rf)

roc_auc_lr = auc(fpr_lr, tpr_lr)
roc_auc_knn = auc(fpr_knn, tpr_knn)
roc_auc_svm = auc(fpr_svm, tpr_svm)
roc_auc_nb = auc(fpr_nb, tpr_nb)
roc_auc_dtc = auc(fpr_dtc, tpr_dtc)
roc_auc_rf = auc(fpr_rf, tpr_rf)

In [ ]:
plt.figure(figsize=(14, 7))

plt.suptitle("ROC",fontsize=24)
plt.subplots_adjust(wspace = 0.4, hspace= 0.4)


plt.subplot(2,3,1)
plt.title("Logistic Regression ROC")
plot_roc_curve(fpr_lr, tpr_lr)
plt.plot(fpr_lr, tpr_lr, label='AUC = %0.4f'% roc_auc_lr)
plt.legend(loc='lower right')

plt.subplot(2,3,2)
plt.title("K Nearest Neighbors ROC")
plot_roc_curve(fpr_knn, tpr_knn)
plt.plot(fpr_knn, tpr_knn, label='AUC = %0.4f'% roc_auc_knn)
plt.legend(loc='lower right')

plt.subplot(2,3,3)
plt.title("Support Vector Machine ROC")
plot_roc_curve(fpr_svm, tpr_svm)
plt.plot(fpr_svm, tpr_svm, label='AUC = %0.4f'% roc_auc_svm)
plt.legend(loc='lower right')

plt.subplot(2,3,4)
plt.title("Naive Bayes ROC")
plot_roc_curve(fpr_nb, tpr_nb)
plt.plot(fpr_nb, tpr_nb, label='AUC = %0.4f'% roc_auc_nb)
plt.legend(loc='lower right')

plt.subplot(2,3,5)
plt.title("Decision Tree Classifier ROC")
plot_roc_curve(fpr_dtc, tpr_dtc)
plt.plot(fpr_dtc, tpr_dtc, label='AUC = %0.4f'% roc_auc_dtc)
plt.legend(loc='lower right')

plt.subplot(2,3,6)
plt.title("Random Forest")
plot_roc_curve(fpr_rf, tpr_rf)
plt.plot(fpr_rf, tpr_rf, label='AUC = %0.4f'% roc_auc_rf)
plt.legend(loc='lower right')

In [ ]:
plt.figure(figsize=(24,12))

plt.suptitle("Confusion Matrixes",fontsize=24)
plt.subplots_adjust(wspace = 0.4, hspace= 0.4)

plt.subplot(2,3,1)
plt.title("Logistic Regression Confusion Matrix")
sns.heatmap(cm_lr,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.subplot(2,3,2)
plt.title("K Nearest Neighbors Confusion Matrix")
sns.heatmap(cm_knn,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.subplot(2,3,3)
plt.title("Support Vector Machine Confusion Matrix")
sns.heatmap(cm_svm,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.subplot(2,3,4)
plt.title("Naive Bayes Confusion Matrix")
sns.heatmap(cm_nb,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.subplot(2,3,5)
plt.title("Decision Tree Classifier Confusion Matrix")
sns.heatmap(cm_dtc,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.subplot(2,3,6)
plt.title("Random Forest Confusion Matrix")
sns.heatmap(cm_rf,annot=True,cmap="Blues",fmt="d",cbar=False, annot_kws={"size": 24})

plt.show()